#  <center> Taller  de Aprendizaje Automático </center>
##  <center> PROYECTO 2: Freesound Audio Tagging 2019  </center>
###  <center> Grupo C  </center>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import os
import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics
import pandas as pd
import shutil
import wave
from IPython.display import Audio
from scipy.io import wavfile
from scipy.signal import resample
from tensorflow.keras.models import Sequential
import pydub
from pydub.silence import split_on_silence
from pydub import AudioSegment, effects
from scipy.io.wavfile import read, write
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from comet_ml import Experiment
from tensorflow.keras.models import load_model # model = load_model("modelo_final.keras", custom_objects={'LWLRAP': LWLRAP})
import kapre
from spec_augment import SpecAugment # https://github.com/MichaelisTrofficus/spec_augment
from skmultilearn.model_selection import iterative_train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping


C:\Users\ureca\miniconda3\envs\tf-gpu\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# Carga de datos

In [ ]:
# df_train_curados = pd.read_csv('train_curated.csv')
# df_train_ruidosos = pd.read_csv('train_noisy.csv')

df_train_curados = pd.read_csv('train_curated.csv')
df_train_ruidosos = pd.read_csv('train_noisy.csv')

df_test = pd.read_csv('sample_submission_v24.csv')


# Funciones de preprocesamiento

## De carpetas

Se dejan por si las quieren ver pero se ejecutaron sobre las carpetas no en este notebook

In [ ]:
# sacar silencios
# https://onkar-patil.medium.com/how-to-remove-silence-from-an-audio-using-python-50fd2c00557d

def sacar_silencios(ruta_audio, nombre,path_salida, largo_optimo = 1000):
    ''''
    ruta_audio: ruta del audio a recortar
    largo_optimo: largo en mili segundos a considerar
    '''

    rate, audio = read(ruta_audio)

    largo_audio = len(audio)/rate *1000

    largo = min(largo_optimo,largo_audio) # se usa solo para curados

    aud = AudioSegment(audio.tobytes(),frame_rate = rate,
                        sample_width = audio.dtype.itemsize,channels = 1)

    audio_chunks = split_on_silence(
        aud,
        # USADO PARA CURADOS
        # min_silence_len = int(largo/10), # evaluar si 1/4 del largo optimo esta ok o no. a lo mejor
        # silence_thresh = -45, # umbral de silencio.. averiguar ruidos tipicos
        # keep_silence = largo/100,) # o menos? ver que sea en funcion del largo del audio

        # USADO PARA RUIDOSOS
        min_silence_len=300,
        silence_thresh=-40,
        keep_silence=100)


    # esto es para cuando todo es detectado como silencio
    if len(audio_chunks) == 0:
        audio_chunks = [aud]
    audio_processed = sum(audio_chunks)
    audio_processed = np.array(audio_processed.get_array_of_samples())

    ruta_audio_salida = path_salida + nombre

    write(ruta_audio_salida,rate,audio_processed)

In [ ]:
def resampleo(ruta_audio,nombre,path_salida):
    rate, audio = read(ruta_audio)
    origin_num_samples = len(audio)
    new_samps = int(origin_num_samples * 16000/rate)

    # resampling
    target_audio_scipy = resample(audio, new_samps).astype(int)
    target_audio_scipy = np.array(target_audio_scipy, np.int16)

    ruta_audio_salida = path_salida + nombre # evaluar nombre y si lo guardamos en la misma carpeta

    write(ruta_audio_salida,16000,target_audio_scipy)
    return target_audio_scipy

## Del pipeline - TF

In [ ]:
# llevar todos los audios al mismo largo -> estrategia: repetir secuencia hasta obtener largo definido como óptimo
# cortar los largos para que en cada batch elija una parte diferente cada vez

def energia(waveform):
    '''
    Calcula la energía de un waveform como la suma de sus muestras al cuadrado
    '''
    return tf.reduce_sum(tf.square(waveform))


def largo_fijo(ruta_audio, largo_optimo = 3):
    ''''
    ruta_audio: ruta del audio a recortar
    largo_optimo: largo en segundos a considerar
    '''
    file = tf.io.read_file(ruta_audio)
    waveform, sample_rate = tf.audio.decode_wav(file)  # sample_rate correcto

    waveform = tf.squeeze(waveform, axis=-1)  #  1D

    wav_len = tf.shape(waveform)[0]
    largo_optimo_samples = tf.cast(sample_rate * largo_optimo, tf.int32)

    if tf.less(wav_len, largo_optimo_samples):
        # print('entro bien (audio corto)')
        reps = tf.cast(
            tf.math.ceil(tf.cast(largo_optimo_samples, tf.float32) / tf.cast(wav_len, tf.float32)),
            tf.int32
        )
        waveform = tf.tile(waveform, [reps])
        waveform = waveform[:largo_optimo_samples]

    elif tf.greater(wav_len, largo_optimo_samples):
        inicio = tf.random.uniform([], 0, wav_len - largo_optimo_samples + 1, dtype=tf.int32, seed=42)
        waveform_i = waveform[inicio:inicio + largo_optimo_samples]
        for _ in range(2):
          inicio2 = tf.random.uniform([], 0, wav_len - largo_optimo_samples + 1, dtype=tf.int32, seed=42)
          waveform2 = waveform[inicio2:inicio2 + largo_optimo_samples]
          if energia(waveform2) > energia(waveform_i):
              waveform_i = waveform2

        waveform = waveform_i  # usar el segmento de mayor energía

    # Si es igual, no hago nada
    # else:
    #     print('dura 1 segundo')

    return waveform, sample_rate

In [ ]:
def audio_a_imagen(waveform, sample_rate):
    frame_length = 1024
    frame_step = 512
    window_fn = tf.signal.hann_window
    fft_length = 1024

    stft = tf.signal.stft(waveform,frame_length = frame_length,frame_step = frame_step,fft_length = fft_length,window_fn = window_fn)
    spectogram = tf.abs(stft)
    num_spectogram_bins = stft.shape[-1]

    lower_edge_hertz, upper_edge_hertz, num_mel_bins= 0, 4000, 80 # evaluar valor frec max por largo imagen
    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(num_mel_bins,num_spectogram_bins,sample_rate,lower_edge_hertz,upper_edge_hertz)

    # se aplica el banco de filtros sobre el espectograma
    mel_spectograms = tf.tensordot(spectogram,linear_to_mel_weight_matrix,1)
    mel_spectograms.set_shape(spectogram.shape[:-1].concatenate(linear_to_mel_weight_matrix.shape[-1:]))

    # calculo el espectograma en magnitud logaritmica y escala mel
    log_mel_spectograms = tf.math.log(mel_spectograms+1e-6)

    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectograms)[...,:13]

    ##################### usamos valores generales de log mel (buscar bibliografia de referencia)

    log_mel_db = 20 * log_mel_spectograms / tf.math.log(10.0) # lo llevo a decibeles para usar valores de referencia

    min_val = -80
    max_val = 0
    log_mel_db = tf.clip_by_value(log_mel_db, min_val, max_val)  # llevo todo al rango -80 0
    log_mel_reescalado = (log_mel_db - min_val) / (max_val - min_val) * 255.0 # llevo todo a 0 - 255

    return log_mel_reescalado

In [ ]:
def codificar_etiquetas(df, columna='labels'):
    # todas las etiquetas ordenadas alfabeticamente
    all_labels = sorted(set(','.join(df[columna]).split(',')))
    label_to_index = {label: i for i, label in enumerate(all_labels)}
    num_classes = len(all_labels)
    # funcion adaptada del ejemplo
    def encode_labels(label_str):
        vec = np.zeros(num_classes, dtype=np.float32)
        for label in label_str.split(','):
            vec[label_to_index[label]] = 1.0
        return vec
    # mapeo
    df = df.copy()
    df['label_vec'] = df[columna].map(encode_labels)

    return df

# METRICA

https://www.kaggle.com/code/carlthome/l-lrap-metric-for-tf-keras

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import label_ranking_average_precision_score

def _lwlrap_sklearn(truth, scores):
    """Reference implementation from https://colab.research.google.com/drive/1AgPdhSp7ttY18O3fEoHOQKlt_3HJDLi8"""
    sample_weight = np.sum(truth > 0, axis=1)
    nonzero_weight_sample_indices = np.flatnonzero(sample_weight > 0)
    overall_lwlrap = label_ranking_average_precision_score(
        truth[nonzero_weight_sample_indices, :] > 0,
        scores[nonzero_weight_sample_indices, :],
        sample_weight=sample_weight[nonzero_weight_sample_indices])
    return overall_lwlrap


def _one_sample_positive_class_precisions(example):
    y_true, y_pred = example

    retrieved_classes = tf.argsort(y_pred, direction='DESCENDING')
    class_rankings = tf.argsort(retrieved_classes)
    retrieved_class_true = tf.gather(y_true, retrieved_classes)
    retrieved_cumulative_hits = tf.math.cumsum(tf.cast(retrieved_class_true, tf.float32))

    idx = tf.where(y_true)[:, 0]
    i = tf.boolean_mask(class_rankings, y_true)
    r = tf.gather(retrieved_cumulative_hits, i)
    c = 1 + tf.cast(i, tf.float32)
    precisions = r / c

    dense = tf.scatter_nd(idx[:, None], precisions, [y_pred.shape[0]])
    return dense

class LWLRAP(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name='lwlrap', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes

        self._precisions = self.add_weight(
            name='per_class_cumulative_precision',
            shape=[num_classes],
            initializer='zeros',
        )

        self._counts = self.add_weight(
            name='per_class_cumulative_count',
            shape=[num_classes],
            initializer='zeros',
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        precisions = tf.map_fn(
            fn=_one_sample_positive_class_precisions,
            elems=(y_true, y_pred),
            dtype=(tf.float32),
        )

        increments = tf.cast(precisions > 0, tf.float32)
        total_increments = tf.reduce_sum(increments, axis=0)
        total_precisions = tf.reduce_sum(precisions, axis=0)

        self._precisions.assign_add(total_precisions)
        self._counts.assign_add(total_increments)

    def result(self):
        per_class_lwlrap = self._precisions / tf.maximum(self._counts, 1.0)
        per_class_weight = self._counts / tf.reduce_sum(self._counts)
        overall_lwlrap = tf.reduce_sum(per_class_lwlrap * per_class_weight)
        return overall_lwlrap

    def reset_states(self):
        self._precisions.assign(tf.zeros_like(self._precisions))
        self._counts.assign(tf.zeros_like(self._counts))

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_classes': self.num_classes,
        })
        return config

def test_match_sklearn():

    # Generate dummy data like https://colab.research.google.com/drive/1YwL7ewUE6vSLZRoSf3Oi9efyupLyfcdb
    num_samples = 100
    num_labels = 20
    scores = np.random.rand(num_samples, num_labels)
    truth = np.random.rand(num_samples, num_labels) > 0.5
    truth[0:1, :] = False

    # Compute expected overall LWLRAP.
    desired = _lwlrap_sklearn(truth, scores)

    # Accumulate LWLRAP per minibatch with tf.metrics.Metric.
    metric = LWLRAP(num_labels)
    batch_size = 1
    for i in range(0, num_samples, batch_size):
        y_true = truth[i:i+batch_size]
        y_pred = scores[i:i+batch_size]
        x = metric.update_state(y_true, y_pred)
    actual = metric.result()

    # Make sure both methods get similar averages.
    np.testing.assert_allclose(actual, desired)


if __name__ == '__main__':
    test_match_sklearn()

# FUNCIONES QUE SE USAN EN LOS MODELOS

In [ ]:
#### CONVOLUCIONAL SENCILLO
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    x = layers.Rescaling(1.0 / 255)(inputs)

    x = layers.BatchNormalization()(inputs)
    x = layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.GlobalAveragePooling2D()(x)  # <-- esta capa aplana sin romper la forma
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.1)(x)

    outputs = layers.Dense(num_classes, activation='sigmoid')(x)  # sin activación, logits
    model = keras.Model(inputs, outputs)
    return model

#### CONVOLUCIONAL SENCILLO ADAPTADO PARA SER CABEZA

def make_model_cabeza(x, num_classes):
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(num_classes, activation='sigmoid')(x)
    return outputs

#### RESNET CON CONVOLUCIONAL SENCILLO O DENSAS

def make_resnet50_model_cabeza(input_shape=(224, 224, 3), num_classes=80,capas_entrenables=0,cabeza_basica=False):
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape,
        pooling= None,
    )
    if capas_entrenables == 0:
        base_model.trainable = False  # Congela pesos
    else:
        base_model.trainable = True # habilito todo
        for layer in base_model.layers[:-capas_entrenables]: # deshabilito las ultimas
            layer.trainable = False

    inputs = keras.Input(shape=input_shape)
    # x = layers.Rescaling(1./255)(inputs)  # Normalizar
    x = base_model(inputs, training=(capas_entrenables > 0)) # si entreno capas entonces quiero que haya entrenamiento y da True sino da false
    if cabeza_basica:
        x = layers.GlobalAveragePooling2D()(x)  # pasar a 2D
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(0.3)(x)
        outputs = layers.Dense(num_classes, activation='sigmoid')(x)  # Multilabel
    else:
        outputs= make_model_cabeza(x, num_classes)
    model = keras.Model(inputs, outputs)
    return model

#### MOBILNET CON CONVOLUCIONAL SENCILLO O DENSAS

def make_mobilenet_model_cabeza(input_shape=(224, 224, 3), num_classes=80, capas_entrenables=0, cabeza_basica=False):
    base_model = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape,
        pooling=None
    )
    if capas_entrenables == 0:
        base_model.trainable = False  # Congela pesos
    else:
        base_model.trainable = True # habilito todo
        for layer in base_model.layers[:-capas_entrenables]: # deshabilito las ultimas
            layer.trainable = False

    inputs = keras.Input(shape=input_shape)
    # x = layers.Rescaling(1./255)(inputs)  # Normalizar
    x = base_model(inputs, training=(capas_entrenables > 0)) # si entreno capas entonces quiero que haya entrenamiento y da True sino da false
    if cabeza_basica:
        x = layers.GlobalAveragePooling2D()(x)  # pasar a 2D
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(0.3)(x)
        outputs = layers.Dense(num_classes, activation='sigmoid')(x)  # Multilabel
    else:
        outputs= make_model_cabeza(x, num_classes)
    model = keras.Model(inputs, outputs)
    return model

In [ ]:
# Preprocesamiento basico

def preprocesamiento(ruta_audio,label): # ojo que esta funcion no deja pasar parametros de las funciones que tiene dentro
    '''
    ruta_carpeta: ruta a la carpeta con wav  in silencios y resampleada
    label: posta 80
    '''
    waveform, sample_rate = largo_fijo(ruta_audio)
    log_mel = audio_a_imagen(waveform, sample_rate)
    log_mel = tf.expand_dims(log_mel, axis=-1)
    return log_mel,label

#### preprocesamiento RESNET


def preprocesamiento3canalesresnet(ruta_audio, label):
    waveform, sample_rate = largo_fijo(ruta_audio)
    log_mel = audio_a_imagen(waveform, sample_rate)
    log_mel = tf.expand_dims(log_mel, axis=-1)
    log_mel = tf.image.grayscale_to_rgb(log_mel)
    log_mel = tf.image.resize_with_pad(log_mel, 224, 224)

    log_mel = tf.keras.applications.resnet50.preprocess_input(log_mel)
    return log_mel, label

#### preprocesamiento MobilNet

def preprocesamiento3canalesmobile(ruta_audio, label):
    waveform, sample_rate = largo_fijo(ruta_audio)
    log_mel = audio_a_imagen(waveform, sample_rate)
    log_mel = tf.expand_dims(log_mel, axis=-1)
    log_mel = tf.image.grayscale_to_rgb(log_mel)
    log_mel = tf.image.resize_with_pad(log_mel, 224, 224)

    log_mel = tf.keras.applications.mobilenet_v2.preprocess_input(log_mel)
    return log_mel, label


spec_augment = SpecAugment(freq_mask_param=5,
                           time_mask_param=5,
                           n_freq_mask=1,
                           n_time_mask=1,
                           mask_value=0)

#### preprocesamiento basico aumentado con SpecAugment

def preprocesamiento_aumentado(ruta_audio,label): # ojo que esta funcion no deja pasar parametros de las funciones que tiene dentro
    '''
    ruta_carpeta: ruta a la carpeta con wav sin silencios y resampleada
    label: posta 80
    '''
    log_mel, label = preprocesamiento(ruta_audio,label) # genero log_mel
    log_mel_aumentado = spec_augment(tf.expand_dims(log_mel, 0), training=True) # agrego dim extra para q ande
    log_mel_aumentado_reescalado = tf.squeeze(log_mel_aumentado, axis=0)  # saco dim extra

    return log_mel_aumentado_reescalado,label

#### preprocesamiento ResNet aumentado con SpecAugment

def preprocesamiento3canalesresnet_aumentado(ruta_audio, label):

    log_mel, label = preprocesamiento3canalesresnet(ruta_audio,label) # genero log_mel
    log_mel_aumentado = spec_augment(tf.expand_dims(log_mel, 0), training=True) # agrego dim extra para q ande
    log_mel_aumentado_reescalado = tf.squeeze(log_mel_aumentado, axis=0)  # saco dim extra

    return log_mel_aumentado_reescalado,label

#### preprocesamiento MobilNet aumentado con SpecAugment

def preprocesamiento3canalesmobile_aumentado(ruta_audio, label):

    log_mel, label = preprocesamiento3canalesmobile(ruta_audio,label) # genero log_mel
    log_mel_aumentado = spec_augment(tf.expand_dims(log_mel, 0), training=True) # agrego dim extra para q ande
    log_mel_aumentado_reescalado = tf.squeeze(log_mel_aumentado, axis=0)  # saco dim extra

    return log_mel_aumentado_reescalado,label

In [ ]:
def crear_datasets_train_val(df,carpeta_audio='train_curated_sin_silencios',prep=None,batch_size=32,ratio=0.2,shuffle=True):
    # """
    # prep es comun o resnet
    # """
    df_train_, df_val_ = train_test_split(df, test_size=ratio, shuffle=shuffle, random_state=42)

    df_train_80 = codificar_etiquetas(df_train_)
    df_val_80 = codificar_etiquetas(df_val_)

    carpeta_audio = carpeta_audio+'/' # ojo que acá se agrega la barra

    file_paths_train = [carpeta_audio + fname for fname in df_train_80['fname']]
    file_paths_val = [carpeta_audio + fname for fname in df_val_80['fname']]

    label_vectors_train = df_train_80['label_vec'].tolist()
    label_vectors_val   = df_val_80['label_vec'].tolist()

    ds_train = tf.data.Dataset.from_tensor_slices((file_paths_train, label_vectors_train))
    ds_val   = tf.data.Dataset.from_tensor_slices((file_paths_val, label_vectors_val))

    if prep == 'comun':
        ds_train = ds_train.map(preprocesamiento, num_parallel_calls=tf.data.AUTOTUNE)
        ds_val   = ds_val.map(preprocesamiento, num_parallel_calls=tf.data.AUTOTUNE)
    elif prep == 'resnet':
            ds_train = ds_train.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)
            ds_val   = ds_val.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)
    elif prep == 'mobile':
        ds_train = ds_train.map(preprocesamiento3canalesmobile, num_parallel_calls=tf.data.AUTOTUNE)
        ds_val   = ds_val.map(preprocesamiento3canalesmobile, num_parallel_calls=tf.data.AUTOTUNE)

    ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    ds_val   = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return ds_train, ds_val


def crear_datasets_train_val_strat(df, carpeta_audio='train_curated_sin_silencios_resample', prep=None, batch_size=32, ratio=0.2):
    # """
    # prep es comun o resnet
    # """
    df_codificado = codificar_etiquetas(df)

    X = df_codificado['fname'].values.reshape(-1, 1)  # reshape porque X debe ser 2D
    y = np.stack(df_codificado['label_vec'].values)   # convertir lista de vectores a array

    X_train, y_train, X_val, y_val = iterative_train_test_split(X, y, test_size=ratio)

    # Reconstruir DataFrames
    df_train_ = pd.DataFrame({'fname': X_train.ravel(), 'label_vec': list(y_train)})
    df_val_   = pd.DataFrame({'fname': X_val.ravel(),   'label_vec': list(y_val)})

    carpeta_audio = carpeta_audio + '/'

    file_paths_train = [carpeta_audio + fname for fname in df_train_['fname']]
    file_paths_val   = [carpeta_audio + fname for fname in df_val_['fname']]

    ds_train = tf.data.Dataset.from_tensor_slices((file_paths_train, df_train_['label_vec'].tolist()))
    ds_val   = tf.data.Dataset.from_tensor_slices((file_paths_val, df_val_['label_vec'].tolist()))

    if prep == 'comun':
        ds_train = ds_train.map(preprocesamiento, num_parallel_calls=tf.data.AUTOTUNE)
        ds_val   = ds_val.map(preprocesamiento, num_parallel_calls=tf.data.AUTOTUNE)
    elif prep == 'resnet':
        ds_train = ds_train.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)
        ds_val   = ds_val.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)
    elif prep == 'mobile':
        ds_train = ds_train.map(preprocesamiento3canalesmobile, num_parallel_calls=tf.data.AUTOTUNE)
        ds_val   = ds_val.map(preprocesamiento3canalesmobile, num_parallel_calls=tf.data.AUTOTUNE)
    elif prep == 'modelo3':
        ds_train = ds_train.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)
        ds_val   = ds_val.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)

    ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    ds_val   = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return ds_train, ds_val

def crear_datasets_train_val_aumentado_strat(df,carpeta_audio=None,prep=None,batch_size=32,ratio=0.2):
    # """
    # prep es comun o resnet
    # """
    df_codificado = codificar_etiquetas(df)

    X = df_codificado['fname'].values.reshape(-1, 1)  # reshape porque X debe ser 2D
    y = np.stack(df_codificado['label_vec'].values)   # convertir lista de vectores a array

    X_train, y_train, X_val, y_val = iterative_train_test_split(X, y, test_size=ratio)

    # Reconstruir DataFrames
    df_train_ = pd.DataFrame({'fname': X_train.ravel(), 'label_vec': list(y_train)})
    df_val_   = pd.DataFrame({'fname': X_val.ravel(),   'label_vec': list(y_val)})

    carpeta_audio = carpeta_audio+'/' # ojo que acá se agrega la barra

    file_paths_train = [carpeta_audio + fname for fname in df_train_['fname']]
    file_paths_val = [carpeta_audio + fname for fname in df_val_['fname']]

    label_vectors_train = df_train_['label_vec'].tolist()
    label_vectors_val   = df_val_['label_vec'].tolist()

    ds_train = tf.data.Dataset.from_tensor_slices((file_paths_train, label_vectors_train))
    ds_train_aumentado = tf.data.Dataset.from_tensor_slices((file_paths_train, label_vectors_train))
    ds_val   = tf.data.Dataset.from_tensor_slices((file_paths_val, label_vectors_val))


    if prep == 'comun':
        ds_train = ds_train.map(preprocesamiento, num_parallel_calls=tf.data.AUTOTUNE)
        ds_train_aumentado = ds_train_aumentado.map(preprocesamiento_aumentado, num_parallel_calls=tf.data.AUTOTUNE)
        ds_val   = ds_val.map(preprocesamiento, num_parallel_calls=tf.data.AUTOTUNE)
    elif prep == 'resnet':
            ds_train = ds_train.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)
            ds_train_aumentado = ds_train_aumentado.map(preprocesamiento3canalesresnet_aumentado, num_parallel_calls=tf.data.AUTOTUNE)
            ds_val   = ds_val.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)
    elif prep == 'mobile':
            ds_train = ds_train.map(preprocesamiento3canalesmobile, num_parallel_calls=tf.data.AUTOTUNE)
            ds_train_aumentado = ds_train_aumentado.map(preprocesamiento3canalesmobile_aumentado, num_parallel_calls=tf.data.AUTOTUNE)
            ds_val   = ds_val.map(preprocesamiento3canalesmobile, num_parallel_calls=tf.data.AUTOTUNE)
    elif prep == 'modelo3':
            ds_train = ds_train.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)
            ds_train_aumentado = ds_train_aumentado.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)

            mixup_layer = tf.keras.layers.MixUp(alpha=0.3)

            def to_dict(x, y):
                return {"images": x, "labels": y}

            def from_dict(batch):
                return batch["images"], batch["labels"]

            ds_train_aumentado = ds_train_aumentado.batch(batch_size)
            ds_train_aumentado = ds_train_aumentado.map(to_dict)
            ds_train_aumentado = ds_train_aumentado.map(lambda batch: mixup_layer(batch, training=True))
            ds_train_aumentado = ds_train_aumentado.map(from_dict)
            ds_train_aumentado = ds_train_aumentado.prefetch(tf.data.AUTOTUNE)


            ds_val   = ds_val.map(preprocesamiento3canalesresnet, num_parallel_calls=tf.data.AUTOTUNE)

            ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
            ds_train = ds_train.concatenate(ds_train_aumentado)
            ds_val   = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    if prep != 'modelo3':
        ds_train = ds_train.concatenate(ds_train_aumentado) # junto los dos train

        ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        ds_val   = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return ds_train, ds_val

In [ ]:
def entrenar_para_test(modelo, df_train, path_audios, batch_size=128, epocas=50, df_val=None):
    """
    Dejo la opcion de val por si usamos earlystopping o similar
    """

    file_paths_train = [path_audios + fname for fname in df_train['fname']]

    df_train_80 = codificar_etiquetas(df_train)
    label_vectors_train = df_train_80['label_vec'].tolist()

    ds_train = tf.data.Dataset.from_tensor_slices((file_paths_train, label_vectors_train))
    ds_train = ds_train.map(lambda path, label: preprocesamiento(path, label), num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    if df_val is not None:
        file_paths_val = [path_audios + fname for fname in df_val['fname']]
        df_val_80 = codificar_etiquetas(df_val)
        label_vectors_val = df_val_80['label_vec'].tolist()

        ds_val = tf.data.Dataset.from_tensor_slices((file_paths_val, label_vectors_val))
        ds_val = ds_val.map(lambda path, label: preprocesamiento(path, label), num_parallel_calls=tf.data.AUTOTUNE)
        ds_val = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)

        history = modelo.fit(ds_train, validation_data=ds_val, epochs=epocas)
    else:
        history = modelo.fit(ds_train, epochs=epocas)

    return history


def entrenar_para_test_resnet(modelo, df_train, path_audios, batch_size=128, epocas=50, df_val=None):
    """
    Dejo la opcion de val por si usamos earlystopping o similar
    """

    file_paths_train = [path_audios + fname for fname in df_train['fname']]

    df_train_80 = codificar_etiquetas(df_train)
    label_vectors_train = df_train_80['label_vec'].tolist()

    ds_train = tf.data.Dataset.from_tensor_slices((file_paths_train, label_vectors_train))
    ds_train = ds_train.map(lambda path, label: preprocesamiento3canalesresnet(path, label), num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    if df_val is not None:
        file_paths_val = [path_audios + fname for fname in df_val['fname']]
        df_val_80 = codificar_etiquetas(df_val)
        label_vectors_val = df_val_80['label_vec'].tolist()

        ds_val = tf.data.Dataset.from_tensor_slices((file_paths_val, label_vectors_val))
        ds_val = ds_val.map(lambda path, label: preprocesamiento3canalesresnet(path, label), num_parallel_calls=tf.data.AUTOTUNE)
        ds_val = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)

        history = modelo.fit(ds_train, validation_data=ds_val, epochs=epocas)
    else:
        history = modelo.fit(ds_train, epochs=epocas)

    return history

def generar_submission_resnet(modelo, df_test, path_salida='submission.csv'):

    file_paths_test = ['test_sin_silencios_resample/' + fname for fname in df_test['fname'].tolist()]
    dummy_labels = [tf.zeros((80,))] * len(file_paths_test)

    ds_test = tf.data.Dataset.from_tensor_slices((file_paths_test, dummy_labels))
    ds_test = ds_test.map(lambda path, label: preprocesamiento3canalesresnet(path, label), num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.batch(32).prefetch(tf.data.AUTOTUNE)

    predicciones = modelo.predict(ds_test)

    labels = df_test.columns[1:].tolist()
    df_submission = pd.DataFrame(predicciones, columns=labels)
    df_submission.insert(0, "fname", df_test["fname"])
    df_submission.to_csv(path_salida, index=False)

    return df_submission

def generar_submission_mobile(modelo, df_test, path_salida='submission.csv'):

    file_paths_test = ['test_sin_silencios_resample/' + fname for fname in df_test['fname'].tolist()]
    dummy_labels = [tf.zeros((80,))] * len(file_paths_test)

    ds_test = tf.data.Dataset.from_tensor_slices((file_paths_test, dummy_labels))
    ds_test = ds_test.map(lambda path, label: preprocesamiento3canalesmobile(path, label), num_parallel_calls=tf.data.AUTOTUNE)
    ds_test = ds_test.batch(32).prefetch(tf.data.AUTOTUNE)

    predicciones = modelo.predict(ds_test)

    labels = df_test.columns[1:].tolist()
    df_submission = pd.DataFrame(predicciones, columns=labels)
    df_submission.insert(0, "fname", df_test["fname"])
    df_submission.to_csv(path_salida, index=False)

    return df_submission

In [ ]:
def cargar_a_comet(history, exp, optimizer='adam', epochs=50):
    '''
    Ver si queremos cargar el lr, como lo vamos cambiando lo saqué
    '''
    train_loss = history.history.get("loss")
    train_acc = history.history.get("acc")
    train_lwlrap = history.history.get("lwlrap")

    val_loss = history.history.get("val_loss")
    val_acc = history.history.get("val_acc")
    val_lwlrap = history.history.get("val_lwlrap")

    if val_lwlrap:
        best_epoch = np.argmax(val_lwlrap)
    else:
        best_epoch = np.argmin(val_loss)

    best_train_loss = train_loss[best_epoch] if train_loss else None
    best_train_acc = train_acc[best_epoch] if train_acc else None
    best_train_lwlrap = train_lwlrap[best_epoch] if train_lwlrap else None

    best_val_loss = val_loss[best_epoch] if val_loss else None
    best_val_acc = val_acc[best_epoch] if val_acc else None
    best_val_lwlrap = val_lwlrap[best_epoch] if val_lwlrap else None

    for metric_name, values in history.history.items():
        for epoch, value in enumerate(values):
            exp.log_metric(metric_name, value, step=epoch)

    exp.log_metrics({
        "best_train_loss": best_train_loss,
        "best_train_acc": best_train_acc,
        "best_train_lwlrap": best_train_lwlrap,
        "best_val_loss": best_val_loss,
        "best_val_acc": best_val_acc,
        "best_val_lwlrap": best_val_lwlrap,
    })

    exp.log_parameters({
        "optimizer": optimizer,
        "loss": 'binary_crossentropy',
        "epochs": epochs
    })

    exp.end()

# Modelos en test

- Resnet entrenando ultimos dos bloques
- Mobilnet entrenado completamente

Ambos modelos finalizan con la convolucional sencilla

## Resnet entrenando ultimos dos bloques

Si se va a correr considerar cambiar path a archivos al llamar función de crear datasets

In [ ]:
ds_train_ruidosos_resnet, ds_val_ruidosos_resnet = crear_datasets_train_val(df_train_ruidosos,carpeta_audio='train_ruidosos_sin_silencios_resample',prep='resnet',batch_size=32,ratio=0.1,shuffle=True)
ds_train_curados_resnet, ds_val_curados_resnet = crear_datasets_train_val_aumentado_strat(df_train_curados,carpeta_audio='train_curated_sin_silencios_resample',prep='resnet',batch_size=32,ratio=0.1)

In [ ]:
model_res = make_resnet50_model_cabeza(capas_entrenables=94)
model_res.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),loss='binary_crossentropy',metrics=[LWLRAP(num_classes=80)])
model_res.fit(ds_train_ruidosos_resnet, validation_data=ds_val_ruidosos_resnet, epochs=15) #  57 minutos
model_res.save('resnet_primer_entrenamiento.keras')

Epoch 1/15
558/558 [==============================] - ETA: 0s - loss: 0.1030 - lwlrap: 0.1286

c:\Users\Magdalena\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py:2319: UserWarning: Metric LWLRAP implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


558/558 [==============================] - 131s 229ms/step - loss: 0.1030 - lwlrap: 0.1286 - val_loss: 0.0718 - val_lwlrap: 0.2145
Epoch 2/15
558/558 [==============================] - 127s 228ms/step - loss: 0.0708 - lwlrap: 0.2125 - val_loss: 0.0666 - val_lwlrap: 0.2595
Epoch 3/15
558/558 [==============================] - 126s 226ms/step - loss: 0.0672 - lwlrap: 0.2596 - val_loss: 0.0639 - val_lwlrap: 0.3100
Epoch 4/15
558/558 [==============================] - 128s 228ms/step - loss: 0.0648 - lwlrap: 0.2952 - val_loss: 0.0634 - val_lwlrap: 0.3182
Epoch 5/15
558/558 [==============================] - 130s 233ms/step - loss: 0.0631 - lwlrap: 0.3224 - val_loss: 0.0622 - val_lwlrap: 0.3383
Epoch 6/15
558/558 [==============================] - 129s 231ms/step - loss: 0.0618 - lwlrap: 0.3428 - val_loss: 0.0618 - val_lwlrap: 0.3391
Epoch 7/15
558/558 [==============================] - 129s 230ms/step - loss: 0.0608 - lwlrap: 0.3570 - val_loss: 0.0618 - val_lwlrap: 0.3418
Epoch 8/15
558/55

In [ ]:
model_resnet_cargado = keras.models.load_model('resnet_primer_entrenamiento.keras', custom_objects={'LWLRAP': LWLRAP})

In [ ]:
# no dejo nada de resnet entrenable
for layer in model_resnet_cargado.get_layer("resnet50").layers:
    layer.trainable = False

# por si las dudas activo el resto
for layer in model_resnet_cargado.layers:
    if not layer.name.startswith("resnet50"):
        layer.trainable = True

model_resnet_cargado.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),loss='binary_crossentropy',metrics=[LWLRAP(num_classes=80)])

callbacks = [tf.keras.callbacks.ModelCheckpoint(
    filepath='resnet_completo.keras',
    monitor='val_lwlrap',
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)]

history_res = model_resnet_cargado.fit(ds_train_curados_resnet, validation_data=ds_val_curados_resnet, epochs=20,callbacks=callbacks) #  12 minutos

Epoch 1/20
279/279 [==============================] - ETA: 0s - loss: 0.0635 - lwlrap: 0.3321

c:\Users\Magdalena\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py:2319: UserWarning: Metric LWLRAP implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()



Epoch 1: val_lwlrap improved from -inf to 0.42025, saving model to resnet_completo.keras
279/279 [==============================] - 40s 132ms/step - loss: 0.0635 - lwlrap: 0.3321 - val_loss: 0.0542 - val_lwlrap: 0.4202
Epoch 2/20
279/279 [==============================] - ETA: 0s - loss: 0.0554 - lwlrap: 0.4129
Epoch 2: val_lwlrap improved from 0.42025 to 0.46011, saving model to resnet_completo.keras
279/279 [==============================] - 36s 131ms/step - loss: 0.0554 - lwlrap: 0.4129 - val_loss: 0.0505 - val_lwlrap: 0.4601
Epoch 3/20
279/279 [==============================] - ETA: 0s - loss: 0.0515 - lwlrap: 0.4659
Epoch 3: val_lwlrap improved from 0.46011 to 0.48679, saving model to resnet_completo.keras
279/279 [==============================] - 38s 135ms/step - loss: 0.0515 - lwlrap: 0.4659 - val_loss: 0.0487 - val_lwlrap: 0.4868
Epoch 4/20
279/279 [==============================] - ETA: 0s - loss: 0.0490 - lwlrap: 0.5022
Epoch 4: val_lwlrap improved from 0.48679 to 0.51952, 

In [ ]:
best_model_resnet = keras.models.load_model('resnet_completo.keras', custom_objects={'LWLRAP': LWLRAP})

In [ ]:
df_submission_resnet = generar_submission_resnet(best_model_resnet, df_test, path_salida='submission_resnet.csv')

141/141 [==============================] - 14s 96ms/step


## Mobilnet entrenado completamente

In [ ]:
ds_train_ruidosos_M ,ds_val_ruidosos_M = crear_datasets_train_val(df_train_ruidosos,carpeta_audio='train_noisy_sin_silencios',prep='mobile',batch_size=16,ratio=0.2,shuffle=True)

ds_train_curados_M,ds_val_curados_M = crear_datasets_train_val(df_train_curados,carpeta_audio='train_curated_sin_silencios',prep='mobile',batch_size=16,ratio=0.2,shuffle=True)

ds_train_curados_M_aumentado, ds_val_curados_M_aumentado = crear_datasets_train_val_aumentado_strat(df_train_curados,carpeta_audio='train_curated_sin_silencios',prep='mobile',batch_size=16,ratio=0.2)



ds_train_aumentado_M = ds_train_ruidosos_M.concatenate(ds_val_ruidosos_M)
ds_train_aumentado_M=ds_train_aumentado_M.concatenate(ds_train_curados_M)
ds_train_aumentado_M=ds_train_aumentado_M.concatenate(ds_train_curados_M_aumentado)


ds_val_aumentado_M = ds_val_curados_M

In [ ]:
model_mobile = make_mobilenet_model_cabeza(capas_entrenables=156, cabeza_basica = False)
model_mobile.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),loss='binary_crossentropy',metrics=[LWLRAP(num_classes=80)])

model_mobile.fit(ds_train_aumentado_M, validation_data=ds_val_curados_M, epochs=20)
model_mobile.save('mobilenet_entrenamiento.keras')

Epoch 1/20
Instructions for updating:
Use fn_output_signature instead
1984/1984 [==============================] - ETA: 0s - loss: 0.0966 - lwlrap: 0.1422

C:\Users\ureca\miniconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:2319: UserWarning: Metric LWLRAP implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


1984/1984 [==============================] - 275s 133ms/step - loss: 0.0966 - lwlrap: 0.1422 - val_loss: 0.0594 - val_lwlrap: 0.3730
Epoch 2/20
1984/1984 [==============================] - 249s 125ms/step - loss: 0.0675 - lwlrap: 0.2596 - val_loss: 0.0497 - val_lwlrap: 0.5029
Epoch 3/20
1984/1984 [==============================] - 249s 125ms/step - loss: 0.0624 - lwlrap: 0.3281 - val_loss: 0.0430 - val_lwlrap: 0.5805
Epoch 4/20
1984/1984 [==============================] - 251s 126ms/step - loss: 0.0587 - lwlrap: 0.3804 - val_loss: 0.0398 - val_lwlrap: 0.6211
Epoch 5/20
1984/1984 [==============================] - 250s 126ms/step - loss: 0.0559 - lwlrap: 0.4184 - val_loss: 0.0360 - val_lwlrap: 0.6644
Epoch 6/20
1984/1984 [==============================] - 250s 126ms/step - loss: 0.0535 - lwlrap: 0.4488 - val_loss: 0.0332 - val_lwlrap: 0.6921
Epoch 7/20
1984/1984 [==============================] - 249s 126ms/step - loss: 0.0510 - lwlrap: 0.4828 - val_loss: 0.0325 - val_lwlrap: 0.7034
Epo

In [ ]:
best_model_mobile = keras.models.load_model('mobilenet_entrenamiento.keras', custom_objects={'LWLRAP': LWLRAP})

In [ ]:
df_submission_mobile = generar_submission_mobile(best_model_mobile, df_test, path_salida='submission_mobile.csv')

141/141 [==============================] - 16s 111ms/step


In [ ]:
print(len(df_test))

4481


In [ ]:
import pandas as pd


csv_mobile = pd.read_csv("submission_mobile.csv")
csv_resnet = pd.read_csv("submission_resnet.csv")



columnas_numericas = csv_mobile.columns.drop(["fname"])
csv_mobile[columnas_numericas] = csv_mobile[columnas_numericas].astype(float)
csv_resnet[columnas_numericas] = csv_resnet[columnas_numericas].astype(float)


promedio = (csv_mobile[columnas_numericas] + csv_resnet[columnas_numericas]) / 2


promedio.insert(0, "fname", csv_mobile["fname"])
promedio.to_csv("submission_promedio_mobile_resnet.csv", index=False)




MODELO 3 (MixUp)

In [ ]:
input_shape=(224, 224, 3)


filepath('modelo3.keras') #SE PUEDE MODIFICAR


dataset_noisy_train, dataset_noisy_val = crear_datasets_train_val(df_train_ruidosos,carpeta_audio='train_ruidosos_sin_silencios_resample',prep='modelo3',batch_size=32,ratio=0.1,shuffle=True)
dataset_curated_train, dataset_curated_val = crear_datasets_train_val_aumentado_strat(df_train_curados,carpeta_audio='train_curated_sin_silencios_resample',prep='modelo3',batch_size=32,ratio=0.1)



inputs = tf.keras.Input(shape=input_shape)
base_model = tf.keras.applications.ResNet50(weights="imagenet", include_top=False)
x = base_model(inputs)
avg = tf.keras.layers.GlobalAveragePooling2D()(x)
dense1 = tf.keras.layers.Dense(512, activation="relu")(avg)
dropout = tf.keras.layers.Dropout(0.3)(dense1)
dense2 = tf.keras.layers.Dense(256, activation="relu")(dropout)
dropout2 = tf.keras.layers.Dropout(0.3)(dense2)
output = tf.keras.layers.Dense(80, activation="sigmoid")(dropout2)


model = tf.keras.Model(inputs=inputs, outputs=output)

for layer in base_model.layers:
    if "conv4_block" in layer.name or "conv5_block" in layer.name:
        layer.trainable = True
    else:
        layer.trainable = False

lr=0.001

model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
metrics=[LWLRAP(num_classes=80)])

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    monitor='val_lwlrap',
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)


history = model.fit(
    dataset_noisy_train,
    validation_data=dataset_noisy_val,
    epochs=20,
)



for layer in base_model.layers:
    if "conv3_block" in layer.name:
        layer.trainable = True

model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=lr/5),
metrics=[LWLRAP(num_classes=80)])

history2 = model.fit(
    dataset_noisy_train,
    validation_data=dataset_noisy_val,
    epochs=5,
)


for layer in base_model.layers:
  layer.trainable = False



model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(),
metrics=[LWLRAP(num_classes=80)])



history3 = model.fit(
    dataset_curated_train,
    validation_data=dataset_curated_val,
    epochs=20,
    callbacks=[checkpoint_cb]
)
